In [1]:
import transformers
from transformers import pipeline
import pandas as pd
import spacy

In [2]:
#carico il modello per l'italiano
!python -m spacy download it_core_news_lg
nlp = spacy.load("it_core_news_lg")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.9/567.9 MB 2.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
#imposto il modello BERT da usare con la libreria pipeline
unmasker = pipeline('fill-mask', model="dbmdz/bert-base-italian-xxl-cased", tokenizer = "dbmdz/bert-base-italian-xxl-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-base-italian-xxl-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

In [ ]:
#importo le frasi del dataset dal file drive
file = pd.read_excel(r'/content/drive/MyDrive/IOcompletion_tesi_agnese_daffara/Dataset/EXPLICIT_results.xlsx')
sentences = file['sentence_masked']

In [ ]:
#creo due liste per gli output e i prediction scores
output = []
scores = []
for sentence in sentences:
    noun_predictions = []
    #uso l'unmasker per ottenere i primi 10 risultati
    results = unmasker(sentence, top_k=10)
    #per ogni risultato, applico il pos tagging di SpaCy in modo da isolare solo i nomi
    for prediction in results:
        doc = nlp(prediction["token_str"])
        for token in doc:
            if token.pos_ == "NOUN":
                #aggiungo i nomi candidati alla lista noun_predictions
                noun_predictions.append(prediction)
    if noun_predictions:
        #ordino i candidati in base al prediction score
        noun_predictions.sort(key=lambda x: x["score"], reverse=True)
        #isolo il candidato con lo score più alto
        output.append(noun_predictions[0]["token_str"])
        scores.append(round(noun_predictions[0]["score"], 2))
    else:
        output.append("unknown")
        scores.append("unknown")
#stampo output e punteggi
print(output)
print(scores)

[{'score': 0.8984237909317017, 'token': 1938, 'token_str': 'musica', 'sequence': 'Ricordo ancora i tempi in cui ascoltavo la musica dai dischi in vinile.'}, {'score': 0.02089521288871765, 'token': 3641, 'token_str': 'radio', 'sequence': 'Ricordo ancora i tempi in cui ascoltavo la radio dai dischi in vinile.'}, {'score': 0.005096043925732374, 'token': 1505, 'token_str': 'storia', 'sequence': 'Ricordo ancora i tempi in cui ascoltavo la storia dai dischi in vinile.'}, {'score': 0.0047414000146090984, 'token': 11319, 'token_str': 'chitarra', 'sequence': 'Ricordo ancora i tempi in cui ascoltavo la chitarra dai dischi in vinile.'}, {'score': 0.0042443834245204926, 'token': 2439, 'token_str': 'luce', 'sequence': 'Ricordo ancora i tempi in cui ascoltavo la luce dai dischi in vinile.'}, {'score': 0.003812420880421996, 'token': 4416, 'token_str': 'canzone', 'sequence': 'Ricordo ancora i tempi in cui ascoltavo la canzone dai dischi in vinile.'}, {'score': 0.0027507301419973373, 'token': 11810, 't